In [ ]:
#installing the necessary modules to extract data from API
pip install investpy
pip install yfinance
pip install pandas_datareader
pip install pandas_ta

In [ ]:
#importing the necessary modules
from pandas_datareader import data as pdr
import numpy as np
import pandas as pd
import investpy
import yfinance as yf
yf.pdr_override() 

In [ ]:
#get indices from Yahoo Finance API. We will be extracting daily frequency data from 01/01/2010 to 31/12/2020
index_data = yf.download("^GSPC ^DJI ^FTSE ^GDAXI FTSEMIB.MI ^FCHI ^IBEX ^AXJO ^HSI ^N225 ^SSMI ^ISEQ ^OMX ^AEX ^ATX \
       ^BFX  ^BUX 000001.SS ^BSESN ^KS11 ^GSPTSE", start="2010-01-01", end="2020-12-31")

#we will only extract the daily closing price of indices
fundamental_indicator = index_data['Close'].fillna(np.nan)
print(fundamental_indicator.head())

#save the extracted data to csv file
fundamental_indicator.to_csv('fundamental_indicator_yahoo.csv')

In [ ]:
#we need to rename the columns in fundamental_indicators to, for example China index, will be ShanghaiSE-china
index_list = ['ShanghaiSE', 'ftse mib', 'aex', 'atx', 'asx200', 'bfx', 'bsesn', 'bux', 'djia','cac40', 'ftse100', 'dax30', 's&p500', 'S&P/TSX', 'hang seng', 'ibex 35', 'iseq', 'kospi', 'nikkei225', \
              'omx', '^SSMI']
country_list = ['china', 'italy', 'netherlands', 'austria', 'australia', 'belgium', 'india', 'hungary', 'united states', 'france', 'united kingdom', 'germany', 'united states', 'canada', 'hong kong', \
                'spain', 'ireland', 'south korea', 'japan', 'sweden', 'switzerland' ]

column_names = fundamental_indicator.columns
#this renames the column names
for h,i,j in zip(column_names, index_list, country_list):
    fundamental_indicator = fundamental_indicator.rename(columns= { h : i + '_' + j })

In [ ]:
#Since I am unable to obtain Russia, Poland, Denmark and Finland indices from Yahoo Finance API, I will
#obtain them from Investing.com API(investpy)

idx = pd.date_range('2010-01-01', '2020-12-31')
index_list_3 = ['moex', 'wig20', 'omxc20', 'omx helsinki 25' ]
country_list_3 = ['russia', 'poland', 'denmark', 'finland']
for i,j in zip(index_list_3, country_list_3):
    df = investpy.get_index_historical_data(index= i,
                                        country= j,
                                        from_date='01/01/2010',
                                        to_date='31/12/2020')
    df_retained = df['Close']

    #due to indices in Investing.com API not having NaN values for certain days due to public holiday, we need to insert rows with
    #with NaN values in those days
    df_retained.index = pd.DatetimeIndex(df_retained.index)
    df_retained_1 = df_retained.reindex(idx, fill_value=np.nan)

    #this adds the new column to the fundamental_indicator dataframe
    fundamental_indicator[j + '-' + i] = df_retained_1

In [ ]:
#Now we extract exchange rate data from Yahoo Finance API
#Since we are predicting closing price of Switzerland stock index, we will pair CHF against another currency

currency_data = yf.download("NOKCHF=X CHFUSD=X CHFEUR=X CHFAUD=X CHFHKD=X JPYCHF=X CHFDKK=X CHFSEK=X CHFNOK=X CHFPLN=X CHFCZK=X CHFHUF=X, \
                   CHFCNY=X CHFINR=X CHFKRW=X CHFCAD=X", start="2010-01-01", end="2020-12-31")

exchange_rate = currency_data['Close'].fillna(np.nan)
exchange_rate.to_csv('C:/Users/user/Desktop/for udacity stock prediction deep learning project/exchange_rate.csv')

In [ ]:
#due to different public holidays in different countries, we need to reindex the indices, bond yield and
#exchange rate data before concatenating them into a single dataframe

idx = pd.date_range('2010-01-01', '2020-12-31')
indicator_list = [fundamental_indicator, exchange_rate]
for i in indicator_list:
    i.index = pd.DatetimeIndex(i.index)
    i = i.reindex(idx, fill_value=np.nan)

#reindex and fill rows with no data with NaN
fundamental_indicator_2 = fundamental_indicator.reindex(idx, fill_value=np.nan)
exchange_rate_2 = exchange_rate.reindex(idx, fill_value=np.nan)
print(fundamental_indicator_2)
print(exchange_rate_2)

#all the 3 dataset have 4018 rows after reindexing
#concatenate the indices, bond yield and exchange rate data into a single dataframe
all_indicators = pd.concat([fundamental_indicator_2, exchange_rate_2], axis = 1)
print(all_indicators)

#after concatenating, dataframe has 4018 rows and 62 columns.(25 for indices, 21 for exchange rate and
#16 for exchange rate)

In [ ]:
#Now we will be using pandas_ta module to extract technical indicator variables.
#We will be using 3, 5, 7, 10, 15, 20, 25 and 30 days in the past for our technical indicators.

import pandas_ta as ta
length = [3, 5, 7, 10, 15, 20, 25, 30]
#Since we will be forecasting Switzerland stock index, we need to extract Open,Close,High and Low daily price
#which are used to calculate the technical indicators
df_smi = investpy.get_index_historical_data(index= 'smi', country= 'switzerland', from_date='01/01/2010',
                                            to_date='31/12/2020')
df_smi_close = df_smi['Close']
df_smi_high = df_smi['High']
df_smi_low = df_smi['Low']
df_smi_open = df_smi['Open']

#extracting simple moving average
sma_ta = pd.DataFrame()
for i in length:
    sma_ta_1 = ta.sma(df_smi_close, length = i)
    sma_ta['sma_' + str(i) ] = sma_ta_1

#extracting exponential moving average
help(ta.ema)
ema_ta = pd.DataFrame()
for i in length:
    ema_ta_1 = ta.ema(df_smi_close, length = i)
    ema_ta['ema_' + str(i)] = ema_ta_1

#extracting average true range
help(ta.atr)
atr_ta = pd.DataFrame()
for i in length:
    atr_ta_1 = ta.atr(df_smi_high, df_smi_low, df_smi_close, length = i)
    ema_ta['atr_' + str(i)] = atr_ta_1

#extracting relative strength index
help(ta.rsi)
rsi_ta = pd.DataFrame()
for i in length:
    rsi_ta_1 = ta.rsi(df_smi_close, length = i)
    rsi_ta['rsi_' + str(i)] = rsi_ta_1

#extracting average directional index
help(ta.adx)
adx_ta = pd.DataFrame()
for i in length:
    adx_ta_1 = ta.adx(df_smi_high, df_smi_low, df_smi_close, length = i)
    #print(adx_ta_1)
    adx_ta['adx_' + str(i)] = adx_ta_1.iloc[:,0]

#extracting stochastic oscillators
help(ta.stoch)
stoch_k_ta = pd.DataFrame()
stoch_d_ta = pd.DataFrame()
for i in length:
    stoch_ta_1 = ta.stoch(df_smi_high, df_smi_low, df_smi_close, k = i, d = 3, smooth_k = i)
    stoch_k_ta['stoch_k_' + str(i)] = stoch_ta_1.iloc[:,0]
    stoch_d_ta['stoch_d_' + str(i)] = stoch_ta_1.iloc[:,1]

In [ ]:
#concatenate all technical indicator variables into a single dataframe
technical_indicators = pd.concat([sma_ta, ema_ta, atr_ta, rsi_ta, adx_ta, stoch_k_ta, stoch_d_ta], axis = 1)
technical_indicators.to_csv('C:/Users/user/Desktop/for udacity stock prediction deep learning project/technical_indicators_3.csv')

#reindexing the concatenated dataframe
technical_indicators_2 = technical_indicators.reindex(idx, fill_value=np.nan)
#concatenate the previous fundamental indicators with technical indicators
all_indicators_funda_tech = pd.concat([all_indicators, technical_indicators_2], axis = 1)

#convert all variables data type to float
all_indicators_funda_tech_2 = all_indicators_funda_tech.astype(float)
#since first 3 rows of dataframe are all missing values, we exclude them
all_indicators_funda_tech_2 = all_indicators_funda_tech_2.iloc[3:,:]
#to fill missing values, we use linear interpolation since this is sequential data
all_indicators_funda_tech_2 = all_indicators_funda_tech_2.interpolate().ffill().bfill()
#since up to 84 rows for stochastic %D have missing values, causing linear interpolation to fill them with same values,
#I have to remove the first 84 rows
all_indicators_funda_tech_2 = all_indicators_funda_tech_2.iloc[84:-1,:]
all_indicators_funda_tech_2.to_csv('C:/Users/user/Desktop/for udacity stock prediction deep learning project/all_indicators_fundamental_technical_2.csv')

#to remove seasonality and trend from the data, i have to apply logarithm difference
log_return_all_indicators = pd.DataFrame()
column_names_1 = all_indicators_funda_tech_2.columns

log_return_all_indicators = np.log(all_indicators_funda_tech_2).diff()

In [ ]:
# We now extract Switzerland stock index closing price data since this is our target variable
switzerland_index_data = yf.download("^SSMI", start="2010-01-01", end="2020-12-31")
switzerland_index_data_close = switzerland_index_data['Close']
switzerland_index_data_close.to_csv('C:/Users/user/Desktop/for udacity stock prediction deep learning project/switzerland stock index historical data.csv')

#We would include 1, 3 and 5 days for our forecast horizon
switzerland_forecast_horizon_return = pd.DataFrame()
#convert all values to natural log
switzerland_index_data_close = np.log(switzerland_index_data_close)
forecast_horizon = [1, 3, 5]

#we need to do 1,3 and 5 days period differencing to get the returns
for i in forecast_horizon:
    switzerland_forecast_horizon_return['forecast horizon_' + str(i) + '_day'] = switzerland_index_data_close.diff(periods = i)

k = 0
for i in forecast_horizon:
    switzerland_forecast_horizon_return.iloc[:,k] = switzerland_forecast_horizon_return.shift(-i)
    k += 1

#drop missing values
switzerland_forecast_horizon_return = switzerland_forecast_horizon_return.dropna()

In [ ]:
#we need to merge the Switzerland stock index return with the independent features.
all_indicators_with_forecast_return =pd.merge(log_return_all_indicators, switzerland_forecast_horizon_return, how="right",
    on=None, left_on=None, right_on=None, left_index=True, right_index=True, sort=True, suffixes=("_x", "_y"), copy=True,
    indicator=False, validate=None)
all_indicators_with_forecast_return = all_indicators_with_forecast_return.dropna()
forecast_return_classification = all_indicators_with_forecast_return.iloc[:,-3:]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#we would need to split the dataset into 75% training data and 25% test data
total_rows = all_indicators_with_forecast_return.shape[0]
training_size = int(0.75 * total_rows)
X_train = all_indicators_with_forecast_return.iloc[:training_size,:-3]
X_test = all_indicators_with_forecast_return.iloc[training_size:,:-3]

#we need to convert the returns for our target variables into binary values: 1 for upward movement
#and 0 for downward movement
forecast_return_classification_binary = ((forecast_return_classification.values) > 0).astype(int)

#we then split the target variable into y_train and y_test
y_train = forecast_return_classification_binary[:training_size,:]
y_test = forecast_return_classification_binary[training_size:,:]

In [ ]:
#To find the 10 most important features, we will use chi squared test and mutual info classification
from sklearn.feature_selection import chi2, mutual_info_regression, mutual_info_classif
from sklearn.feature_selection import SelectKBest

selected_features = SelectKBest(mutual_info_classif, k=10).fit(X_train, y_train[:,2])
column_names = X_train.columns
interesting_features_df = pd.DataFrame(selected_features.scores_, index=column_names).rename(columns={0:'feature weight'})

#this sorts the features from highest weight to lowest weight
interesting_features_df = interesting_features_df.sort_values('feature weight', ascending=False)

#this displays the 10 most important features in horizontal bar chart and the corresponding weights
import matplotlib.pyplot as plt
Features = interesting_features_df.index[0:10].to_list()[::-1]
Weight = interesting_features_df['feature weight'][0:10].to_list()[::-1]

plt.barh(Features, Weight, color='orange')
plt.title('10 MOST IMPORTANT FEATURES')
plt.ylabel('FEATURES')
plt.xlabel('WEIGHT')
plt.show()

In [ ]:
#this is the chi squared test to find the 10 most important features
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

selected_features = SelectKBest(chi2, k=10).fit(X_train_scaled, y_train[:,2])
interesting_features_df = pd.DataFrame(selected_features.scores_, index=column_names).rename(columns={0:'feature weight'})

#this sorts the features from highest weight to lowest weight
interesting_features_df = interesting_features_df.sort_values('feature weight', ascending=False)

#this displays the 10 most important features in horizontal bar chart and the corresponding weights
import matplotlib.pyplot as plt
Features = interesting_features_df.index[0:10].to_list()[::-1]
Weight = interesting_features_df['feature weight'][0:10].to_list()[::-1]

plt.barh(Features, Weight, color='orange')
plt.title('10 MOST IMPORTANT FEATURES')
plt.ylabel('FEATURES')
plt.xlabel('WEIGHT')
plt.show()

In [ ]:
#importing the necessary modules for bayesian optimization
import ast
import csv
import inspect
import sys
import time
from time import gmtime, strftime
from timeit import default_timer as timer
import numpy as np
import pandas as pd
import xgboost as xgb
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, f1_score
import pickle
import tensorflow_addons as tfa

In [ ]:
results_file = "bayes_optimization_summary_{}.csv".format(
    strftime("%y%m%d%H%M", gmtime())
)

#used to name the csv file for recording accuracy and F1 scores and hyperparameters combination
with open(results_file, "w", newline="") as outfile:
    writer = csv.writer(outfile)
    writer.writerow(
        [
            "iteration",
            "loss",
            "accuracy_score",
            "f1_score"
            "train_time",
            "params",
        ]
    )

In [ ]:
def objective(space):

    global ITERATION
    ITERATION += 1
    start = timer()
    global predictions_all

    
    #train the XGBClassifier model on training data and test data is used as validation data with early stopping round set at 40
    eval_set = [(X_test, y_test[:,2])]
    xgbcl = xgb.XGBClassifier(colsample_bytree =  space['colsample_bytree'], gamma = space['gamma'], learning_rate = space['learning_rate'], max_depth = space['max_depth'], min_child_weight = space['min_child_weight'], n_estimators = space['n_estimators'], reg_alpha = space['reg_alpha'], scale_pos_weight = space['scale_pos_weight'], subsample = space['subsample'], tree_method = 'gpu_hist', sampling_method = 'gradient_based')
    xgbcl.fit(X_train, y_train[:,2], early_stopping_rounds=40, eval_metric="error@0.55", eval_set=eval_set)
    #the trained model is used to predict on the test data
    prediction = xgbcl.predict(X_test)
    predictions_all = pd.concat([predictions_all, pd.DataFrame(prediction).rename(columns={0:ITERATION})], axis=1)
    #obtain the accuracy and f1 score
    acc = accuracy_score(y_test[:,2], prediction)
    f1 = f1_score(y_test[:,2], prediction) 

    # log runtime
    run_time = timer() - start

    # calculate loss based on scoring method
    loss = 1 - acc
    
    # export results to csv
    out_file = results_file
    with open(out_file, "a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(
            [
                ITERATION,
                loss,
                acc,
                f1,
                run_time,
                space
            ]
        )

    return loss

In [ ]:
#this is the hyperparameter search space for XGBoost Classifier
space = { 'learning_rate': hp.uniform('learning_rate', 0.001, 0.050),
'max_depth': hp.choice('max_depth', np.arange(4, 14, 1, dtype=int)),
'min_child_weight': hp.choice('min_child_weight', np.arange(1, 20, 1, dtype = int)),
'gamma': hp.uniform('gamma', 0.04, 0.4),
'subsample': hp.uniform('subsample', 0.5, 1.0),
'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0),
'reg_alpha': hp.uniform('reg_alpha', 0.002 , 0.04),
'n_estimators': hp.choice('n_estimators', np.arange(200, 500,1, dtype = int)),
'scale_pos_weight': hp.choice('scale_pos_weight', np.arange(10, 50, 1, dtype=int)),
'tree_method' :'gpu_hist',
'sampling_method' : 'gradient_based'
          }

In [ ]:
#this line of codes initiates the bayesian optimisation procedure and I will set maximum iterations at 400
ITERATION=0
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=400,
    trials=Trials(),
    show_progressbar=True,
)